## Speech recognition using DeepSpeech

In [ ]:
from pathlib import Path

In [ ]:
audio_directory = Path.cwd() / 'audio'
raw_text_directory = Path.cwd() / 'raw_text'
normal_text_directory = Path.cwd() / 'normal_text'

In [ ]:
# deepspeech --model deepspeech-0.9.3-models.pbmm --scorer deepspeech-0.9.3-models.scorer --audio audio/2830-3980-0043.wav
# also
# beam_width # Beam width for the CTC decoder
# lm_alpha # Language model weight (lm_alpha). If not specified, use default from the scorer package.
# lm_beta # Word insertion bonus (lm_beta). If not specified, use default from the scorer package.
# parser.add_argument('--version', action=VersionAction,
#                     help='Print version and exits')
# parser.add_argument('--extended', required=False, action='store_true',
#                     help='Output string from extended metadata')
# parser.add_argument('--json', required=False, action='store_true',
#                     help='Output json from metadata with timestamp of each word')
# parser.add_argument('--candidate_transcripts', type=int, default=3,
#                     help='Number of candidate transcripts to include in JSON output')
# parser.add_argument('--hot_words', type=str,
#                     help='Hot-words and their boosts.')


# Inference took 1259.075s for 4456.333s audio file.

In [ ]:

print(f"Loading model from file {model_file}")
model_load_start = timer()

# Creating a model instance and loading model
ds = Model(model_file)

model_load_end = timer() - model_load_start
print('Loaded model in {:.3}s.'.format(model_load_end), file=sys.stderr)

# if args.beam_width:
    # ds.setBeamWidth(args.beam_width)

desired_sample_rate = ds.sampleRate()

if scorer_file:
    print(f"Loading scorer from files {scorer_file}")
    scorer_load_start = timer()
    ds.enableExternalScorer(scorer_file)
    scorer_load_end = timer() - scorer_load_start
    print(f"Loaded scorer in {scorer_load_end:.3}s.")

    # if args.lm_alpha and args.lm_beta:
    #     ds.setScorerAlphaBeta(args.lm_alpha, args.lm_beta)

# if args.hot_words:
#     print('Adding hot-words', file=sys.stderr)
#     for word_boost in args.hot_words.split(','):
#         word,boost = word_boost.split(':')
#         ds.addHotWord(word,float(boost))

fin = wave.open(audio_file, 'rb')
fs_orig = fin.getframerate()
if fs_orig != desired_sample_rate:
    print(f"Warning: original sample rate ({fs_orig}) is different than {desired_sample_rate}hz. Resampling might produce erratic speech recognition.")
    fs_new, audio = convert_samplerate(audio_file, desired_sample_rate)
else:
    audio = np.frombuffer(fin.readframes(fin.getnframes()), np.int16)

audio_length = fin.getnframes() * (1/fs_orig)
fin.close()



# Performing inference
print("Running inference.")
inference_start = timer()
if True:
    print(metadata_to_string(ds.sttWithMetadata(audio, 1).transcripts[0]))
elif args.json:
    print(metadata_json_output(ds.sttWithMetadata(audio, args.candidate_transcripts)))
else:
    print(ds.stt(audio))
inference_end = timer() - inference_start
print(f"Inference took {inference_end:0.3f}s for {audio_length:0.3f}s audio file.")